# Hashtags

In [1]:
from nltk.tokenize import TweetTokenizer
import os
import pandas as pd
import re
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from IPython.display import clear_output

In [2]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [7]:
DATADIR = "../data/text/"
ID_STR = "id_str"
TEXT = "text"
TOPICQUERY = "corona|covid|huisarts|mondkapje|rivm|blijfthuis|flattenthecurve|houvol"
PANDEMICQUERY = "|".join([TOPICQUERY, r'virus|besmet|ziekenhui|\bic\b|intensive.care|^zorg|vaccin|[^ad]arts|uitbraak|uitbrak|pandemie|ggd|'+
                                      r'mondkapje|quarantaine|\bwho\b|avondklok|variant|verple|sympto|e.golf|mutant|^omt$|umc|hcq|'+
                                      r'hydroxychloroquine|virolo|zkh|oversterfte|patiënt|patient|intensivist|🦠|ivermectin'])
DISTANCEQUERY = "1[.,]5[ -]*m|afstand.*hou|hou.*afstand|anderhalve[ -]*meter"
LOCKDOWNQUERY = "lock.down|lockdown"
VACCINQUERY = "vaccin|ingeënt|ingeent|inent|prik|spuit|bijwerking|-->|💉|pfizer|moderna|astrazeneca|astra|zeneca|novavax|biontech"
TESTQUERY = r'\btest|getest|sneltest|pcr'
QUERY = "|".join([PANDEMICQUERY, TESTQUERY, VACCINQUERY, LOCKDOWNQUERY, DISTANCEQUERY])
BASEQUERY = "corona|covid"

In [19]:
def get_tweets(file_pattern, query, spy=False):
    tweets = []
    file_names = sorted(os.listdir(DATADIR))
    for file_name in file_names:
        if re.search(file_pattern, file_name):
            if spy:
                squeal(file_name)
            df = pd.read_csv(DATADIR+file_name,index_col=ID_STR)
            df_query = df[df[TEXT].str.contains(query, flags=re.IGNORECASE)]
            tweets.extend(list(df_query[TEXT]))
    return(tweets)

In [5]:
def get_hashtags(tweet):
    hashtags = []
    for token in TweetTokenizer().tokenize(tweet):
        if re.search(r'#', token):
            hashtags.append(token)
    return(hashtags)

In [20]:
for month in "202002 202003 202004 202005 202006 202007 202008 202009 202010 202011 202012 201201".split():
    tweets = [re.sub(r'\\n', ' ', tweet) for tweet in get_tweets(month, BASEQUERY, spy=False)]
    hashtags = {}
    for tweet in tweets:
        if re.search(r'#', tweet):
            for hashtag in get_hashtags(tweet):
                if hashtag in hashtags:
                    hashtags[hashtag] += 1
                else:
                    hashtags[hashtag] = 1
    print(month, " ".join([hashtag for hashtag in sorted(hashtags.keys(), key=lambda hashtag:hashtags[hashtag], reverse=True)][:200]))

202002 #coronavirus #corona #Coronavirus #COVID19 #coronavragen #Corona #COVID2019 #COVIDー19 #coronavirusNederland #CoronavirusOutbreak #virus #covid19 #RIVM #China #Coronavirius #FVD #Iran #CoronaVirus #Wuhan #EU #Covid_19 #ncov #COVID #patiënt #schengen #coronavirusus #Tilburg #Covid19 #Coronavius #CoronaVirusUpdates #Nederland #Italie #rivm #covid #dtv #PVV #Italië #china #Nieuwsuur #wuhan #WHO #sarscov2 #jinek #vrtnws #CoronaOutbreak #quarantaine #op1 #handenwassen #nieuws #coronavavirus #dwdd #Jinek #Erdogan #tilburg #coronaviruschina #coronarovirus #coronacrisis #boeren #nepnieuws #nieuwsuur #SARSCoV2 # #GGD #GoogleAlerts #griep #fakenews #COVID_19 #COVID19NL #Coronarivus #opengrenzen #terzaketv #Schengengrenscode #stikstofcrisis #Coronavid19 #waarschuwing #WhatsApp #nepbericht #telegraafpremium #nos #NOS #WuhanVirus #Baudet #coronavrees #Op1 #corona-virus #Rusland #NWO #Europa #vragen #FvD #Diemen #nosjournaal #antwoorden #F1 #vtmnieuws #Westerdam #coronavirus-besmetting #AEX #c